In [ ]:
# Breakdown sentiment by ChatGPT and human
# look at themes and topics of the 30 and 40 slections. Maybe too much emotion or common pattern

###########################################
# Analyzing the number of human responses #
# vs ChatGPT responses selected.          #
###########################################

import json

chatgpt_selected = 0
human_selected = 0

i=0

input_data: list = []

# Open your NDJSON file
with open('humanpref2.ndjson', 'r') as file:
    for line in file:
        try:
            data_dict = json.loads(line)  # Parse each line as JSON

            row_data_str = data_dict.get('data_row', {}).get('row_data', '')

            if row_data_str:
                # Parse the stringified JSON in 'row_data'
                row_data_dict = json.loads(row_data_str)

                # Now extract modelOutputs from the parsed row_data
                model_outputs = row_data_dict['modelOutputs']

                # Save the social media posts for later
                input_data.append(row_data_dict.get('messages')[0]['content'])

                # Identify responses belonging to chatGPT4o and others
                chatgpt4o_response = None
                other_response = None

                for output in model_outputs:
                    if output.get('modelConfigName') == 'ChatGPT4o':
                        chatgpt4o_response = output
                    else:
                        other_response = output


                # If we have both a chatGPT4o response and another response
                if chatgpt4o_response and other_response:
                    # Extract the radio answer
                    id = ""
                    for x in data_dict['projects']:
                        id = x
                        break

                    i+=1

                    classi = data_dict['projects'][id]['labels']

                    if len(classi) > 0:
                        x = classi[0]['annotations']['classifications']

                        if len(x) > 0:
                            radio_answer = x[0].get('radio_answer')

                            # Check if the "name" or "value" matches the chatGPT4o response
                            is_name_match = radio_answer.get('name') == chatgpt4o_response.get('title')[-1]
                            is_value_match = radio_answer.get('value') == chatgpt4o_response.get('title')[-1]

                            if radio_answer.get('name') == chatgpt4o_response.get('title')[-1]:
                                chatgpt_selected += 1
                            if radio_answer.get('name') == other_response.get('title')[-1]:
                                human_selected += 1

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

print("ChatGPT selected ", chatgpt_selected)
print("Human selected ", human_selected)

print(input_data)

In [ ]:
###########################################
# Analyzing the sentiment of each input.  #
#                                         #
###########################################

# Add more than positive and negative

from transformers import pipeline

results: list = []

# Initialize the pipeline
sentiment_analyzer = pipeline('sentiment-analysis')

# Input for analysis
for text in input_data:
    # Perform sentiment analysis

    result = sentiment_analyzer(text)

    results.append(result[0]['label'])

positive_results = results.count("POSITIVE")
negative_results = results.count("NEGATIVE")

print("Positive results: ", positive_results)
print("Negative results: ", negative_results)

for x in results:
    print(x)